In [ ]:
###-------------------------------------------------
### Место, где мы изменяем важные перемнные

DIR_NAME = "Train_before"        # имя папки как она называется на диске
# DATASET_FOLDER = ""       # название архива и одноименной папки в нём где хранятся наши фотографии и аннотации к ним

count_images = 1_000           #  количество фотографий которое мы хотим добавить (если в датасете меньше фотографий, то используется то количество которое имеется)

classes = ["drone"]            # список классов

width = 416                   # ширина и высота изображения (darknet автоматически преобразует фотографии в нужный размер)
height = 416


###---------------------------------
### Advanced level

num_of_clusters = 6    # defolt 6
max_batches = 6000    # or None
improve_find_bounding_boxes = False
subdivisions = 16
random = False

###------------------------------------------------

### Остальное ни в коем случае не трогать


from google.colab import drive
import sys
import os

drive.mount('/content/gdrive')

!ln -s /content/gdrive/MyDrive/ /mydrive # это команда создать ссылку, так что длинный путь /content/gdrive/MyDrive/ будет равнозначен короткому и удобному /mydrive

%cd /mydrive

%cd "$DIR_NAME"



if os.path.exists("darknet") and os.path.isdir("darknet"):

  %cd darknet

else:
  !git clone https://github.com/AlexeyAB/darknet

  %cd darknet

  !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29



!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

!/usr/local/cuda/bin/nvcc --version


if os.path.exists("was_make.txt"):
    print(f"Файл was_make.txt существует.")

else:
    print(f"Файл was_make.txt не существует.")
    print("Компилирую...")
    !make > /dev/null 2>&1
    !touch was_make.txt


print("Компиляция завершена")

%cd ..


# получаем название датасета в рабочей папке
zip_files = [file for file in os.listdir(f"/mydrive/{DIR_NAME}") if file.endswith(".zip")]

# Проверка на отсутствие архивов
if not zip_files:
    print("Нет архивов в указанной папке. Программа остановлена.")
    sys.exit()

# Проверка на наличие нескольких архивов
if len(zip_files) > 1:
    print("Обнаружено несколько архивов. Программа остановлена.")
    sys.exit()


DATASET_FOLDER = zip_files[0][:-4]
print(f"Используется архив: {DATASET_FOLDER}")





# проверка что внутри архива находится одноименная папка

!unzip -l "$DATASET_FOLDER".zip > output.txt     # записываем вывод команды в файл


# Имя архива (без расширения .zip)
archive_name = DATASET_FOLDER  # Замените на ваше значение

# Открываем файл output.txt для чтения
with open(f"/mydrive/{DIR_NAME}/output.txt", "r") as file:
    lines = file.readlines()

# Переменные для проверки
folder_name = None  # Имя папки
files_outside_folder = False  # Флаг для файлов вне папки

# Проходим по строкам файла, начиная с 4-й строки (индекс 3)
for line in lines[3:-2]:
    # отрезаем вначале 
    
    # Archive:  DATASET.zip
    # Length      Date    Time    Name
    # ---------  ---------- -----   ----
    
    
    # а в конце 
    # ---------                     -------
    # 5165007                     189 files



    # Разделяем строку по пробелам и берем последний элемент (имя файла/папки)
    parts = line.split()
    if not parts:
        continue  # Пропускаем пустые строки
    name = parts[-1]  # Последний элемент — это имя файла/папки

    # Проверяем, является ли элемент папкой
    if name.endswith("/"):
        # Если папка уже найдена, это вторая папка — ошибка
        if folder_name is not None:
            print("Ошибка: В архиве больше одной папки.")
            exit(1)

        # Сохраняем имя папки
        folder_name = name.rstrip("/")

    else:
        # Проверяем, что файл находится внутри папки
        if folder_name is None or not name.startswith(folder_name + "/"):
            files_outside_folder = True


# Проверяем, что папка найдена и её имя совпадает с именем архива
if folder_name is None:
    print("Ошибка: В архиве нет папки.")
    sys.exit(1)
elif folder_name != archive_name:
    print(f"Ошибка: Папка '{folder_name}' не совпадает с именем архива '{archive_name}'.")
    sys.exit(1)

# Проверяем, что все файлы находятся внутри папки
if files_outside_folder:
    print("Ошибка: В архиве есть файлы вне папки.")
    sys.exit(1)


# Если все проверки пройдены
print(f"Всё в порядке: В архиве одна папка '{folder_name}', и все файлы находятся внутри неё.")

!rm output.txt

# проверка чтобы лишний раз не тратить время на разархивацию
folder_path = f"darknet/data/{DATASET_FOLDER}"

if os.path.exists(folder_path) and os.path.isdir(folder_path):
    pass
else:
  !unzip "$DATASET_FOLDER".zip -d darknet/data/


%cd darknet/



from random import shuffle

validation_split = 0.2         # 0.2 - это значит, что 20% от всего датасета будет использоваться для валидации (можно менять, но рекомендуется оставить таким)

files = os.listdir(f"data/{DATASET_FOLDER}/")   #папка где хранятся фотографии и аннотации
count_images = min(count_images, (len(files) // 2))

COUNT = count_images
images = []
for file in files:
    # Прерываем цикл, если уже набрали нужное количество изображений
    if count_images <= 0:
        break

    # Проверяем, что файл имеет расширение .jpg или .jpeg
    if file.endswith('.jpg') or file.endswith('.jpeg'):
        images.append(f"data/{DATASET_FOLDER}/" + file)
        count_images -= 1

# Проверяем, сколько изображений было добавлено
if len(images) < COUNT:
    print(f"Предупреждение: Добавлено только {len(images)} изображений (требовалось {COUNT}).")
else:
    print(f"Успешно добавлено {COUNT} изображений.")





shuffle(images)

train_len = int((1-validation_split)*len(images))

train_dataset = images[:train_len]
valid_dataset = images[train_len:]


with open('data/train.txt', 'w') as f:
    txt = '\n'.join(train_dataset)
    f.write(txt)

with open('data/test.txt', 'w') as f:
  txt = '\n'.join(valid_dataset)
  f.write(txt)


classes_amount = len(classes)

with open('data/obj.names', 'w') as f:
  f.write('\n'.join(classes))


# создаём папку для сохранения весов
if os.path.exists(f"/mydrive/{DIR_NAME}/backup") and os.path.isdir(f"/mydrive/{DIR_NAME}/backup"):
  pass
else:
  os.mkdir(f"/mydrive/{DIR_NAME}/backup")


train = 'train.txt'
valid = 'test.txt'
names = 'obj.names'
backup = f'/mydrive/{DIR_NAME}/backup'

txt = f"""classes={classes_amount}
train=data/{train}
valid=data/{valid}
names=data/{names}
backup={backup}"""

with open('data/obj.data', 'w') as f:
  f.write(txt)


classes = classes_amount

filters = (classes + 5) * 3


# Проверяем, было ли значение max_batches указано и больше 6000
if max_batches is not None and max_batches > 6000:
    # Если да, оставляем его
    pass
else:
    # Если нет, рассчитываем max_batches
    max_batches = classes * 3000
    # Убедимся, что max_batches не меньше 6000
    if max_batches < 6000:
        max_batches = 6000
        
        

step1 = int(max_batches * 0.8)
step2 = int(max_batches * 0.9)


def copy_file(file, dest):
  os.system(f'cp {file} -d {dest}')


def change_in_file(file, old, new):
  os.system(f"sed -i 's/^{old}.*/{new}/' {file}")



template_file = "cfg/yolov4-tiny-custom.cfg"      # шаблон нельзя менять
config_file = "cfg/yolov4-tiny-obj.cfg"
max_batches_line = "max_batches = 500200"



copy_file(template_file, config_file)


if improve_find_bounding_boxes:
  change_in_file(config_file, "ignore_thresh", "ignore_thresh = .9")
  change_in_file(config_file, "iou_normalizer", "iou_normalizer=0.5")
  change_in_file(config_file, "iou_loss", "iou_loss=giou")

if subdivisions:
  change_in_file(config_file, "subdivisions", f"subdivisions={subdivisions}")
else:
  change_in_file(config_file, "subdivisions", "subdivisions=1")


change_in_file(config_file, "width", f"width={width}")
change_in_file(config_file, "height", f"height={height}")


if random:
  change_in_file(config_file, "random", "random=1")
else:
  change_in_file(config_file, "random", "random=0")



if num_of_clusters is not None:
  change_in_file(config_file, "num", f"num={num_of_clusters}")
else:
  change_in_file(config_file, "num", f"num=6")


change_in_file(config_file, "steps=400000,450000", f"steps={step1},{step2}")
change_in_file(config_file, "classes=80", f"classes={classes}")
change_in_file(config_file, "filters=255", f"filters={filters}")
change_in_file(config_file, max_batches_line, f"max_batches = {max_batches}")


# пересчет якорей
!./darknet detector calc_anchors data/obj.data -num_of_clusters "$num_of_clusters" -width "$width" -height "$height"

# полученные якоря сохраняются в файле anchors.txt
with open('anchors.txt', 'r', encoding='utf-8') as file:
    anchors = file.read()

change_in_file(config_file, "anchors", f"anchors={anchors}")


# запускаем обучение
!./darknet detector train data/obj.data cfg/yolov4-tiny-obj.cfg yolov4-tiny.conv.29 -map